<a href="https://colab.research.google.com/github/BansiG/CV_Project_Grp3/blob/master/Project%20Code/Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing all the libraries
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import time

In [2]:
#Cloning tensorflow models
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 22599, done.
remote: Total 22599 (delta 0), reused 0 (delta 0), pack-reused 22599
Receiving objects: 100% (22599/22599), 559.99 MiB | 32.19 MiB/s, done.
Resolving deltas: 100% (13412/13412), done.
Checking out files: 100% (2836/2836), done.


In [3]:
#Use of object detection API of tensorflow
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

Selecting previously unselected package libprotobuf10:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libprotobuf10_3.0.0-9.1ubuntu1_amd64.deb ...
Unpacking libprotobuf10:amd64 (3.0.0-9.1ubuntu1) ...
Selecting previously unselected package libprotoc10:amd64.
Preparing to unpack .../libprotoc10_3.0.0-9.1ubuntu1_amd64.deb ...
Unpacking libprotoc10:amd64 (3.0.0-9.1ubuntu1) ...
Selecting previously unselected package libprotobuf-java.
Preparing to unpack .../libprotobuf-java_3.0.0-9.1ubuntu1_all.deb ...
Unpacking libprotobuf-java (3.0.0-9.1ubuntu1) ...
Selecting previously unselected package protobuf-compiler.
Preparing to unpack .../protobuf-compiler_3.0.0-9.1ubuntu1_amd64.deb ...
Unpacking protobuf-compiler (3.0.0-9.1ubuntu1) ...
Setting up libprotobuf-java (3.0.0-9.1ubuntu1) ...
Setting up libprotobuf10:amd64 (3.0.0-9.1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libprotoc10:amd64 (3.0.0-9.1ubuntu1) .

In [4]:
#Initial libraries
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

/content/object_detection/utils/visualization_utils.py:27: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "<decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 87, in catch_config_error
    retu

In [8]:
#This function is used to upload files into google colab repositary
from google.colab import files
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())


upload_files()

Saving test.png to test.png
Saving test1.png to test1.png


['test.png', 'test1.png']

In [0]:
# What model to download.
MODEL_NAME = 'ssd_inception_v2_coco_2017_11_17'

# model with more accurancy but up to you use a diferent model
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_2017_11_08'
# MODEL_NAME = 'faster_rcnn_resnet101_coco_2017_11_08'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
    
#Detection Graph of tensors
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [0]:
#Session run
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [0]:
#This function will make all other pixels black except the region of interest that we have given using
#masking.
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255   
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [43]:
#Read videoframes
video = cv2.VideoCapture("test21.mp4")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = int(video.get(3))
frame_height = int(video.get(4))
#Writing the new video
out = cv2.VideoWriter('output_video.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20 , (frame_width,frame_height))
prev_navigation = 0
curr_navigation = 0
try:
  
  while(video.isOpened()):

      # Acquire frame and expand frame dimensions to have shape: [1, None, None, 3]
      # i.e. a single-column array, where each item in the column has the pixel RGB value
      ret, frame = video.read()
      stime = time.time()
      objects = []
      class_str = ""
      frame_width = frame.shape[0]
      frame_height = frame.shape[1]




      rows, cols = frame.shape[:2]
      #Defining region of interest and Grids for navigation
      left_boundary = [int(cols*0.40), int(rows*0.95)]
      left_boundary_top = [int(cols*0.40), int(rows*0.20)]
      right_boundary = [int(cols*0.60), int(rows*0.95)]
      right_boundary_top = [int(cols*0.60), int(rows*0.20)]    

      up_boundary_left= [int(cols*0.20), int(rows*0.45)]
      up_boundary_right = [int(cols*0.80), int(rows*0.45)]
      down_boundary_left = [int(cols*0.20), int(rows*0.70)]
      down_boundary_right = [int(cols*0.80), int(rows*0.70)]    
      
      
      bottom_left  = [int(cols*0.20), int(rows*0.95)]
      top_left     = [int(cols*0.20), int(rows*0.20)]
      bottom_right = [int(cols*0.80), int(rows*0.95)]
      top_right    = [int(cols*0.80), int(rows*0.20)]
      vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)

#       cv2.line(frame,tuple(bottom_left),tuple(bottom_right), (255, 0, 0), 5)
#       cv2.line(frame,tuple(bottom_right),tuple(top_right), (255, 0, 0), 5)
#       cv2.line(frame,tuple(top_left),tuple(bottom_left), (255, 0, 0), 5)
#       cv2.line(frame,tuple(top_left),tuple(top_right), (255, 0, 0), 5)

      copied = np.copy(frame)
      interested=region_of_interest(copied,vertices)

      frame_expanded = np.expand_dims(interested, axis=0)

      # Perform the actual detection by running the model with the image as input
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: frame_expanded})

      # Draw the results of the detection (aka 'visulaize the results')
      vis_util.visualize_boxes_and_labels_on_image_array(
          frame,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8,
          min_score_thresh=0.78)

      print(frame_width,frame_height)
      
      #get coordinates of the detected objects
      ymin = int((boxes[0][0][0]*frame_width))
      xmin = int((boxes[0][0][1]*frame_height))
      ymax = int((boxes[0][0][2]*frame_width))
      xmax = int((boxes[0][0][3]*frame_height))

      Result = np.array(frame[ymin:ymax,xmin:xmax])

      ymin_str='y min  = %.2f '%(ymin)
      ymax_str='y max  = %.2f '%(ymax)
      xmin_str='x min  = %.2f '%(xmin)
      xmax_str='x max  = %.2f '%(xmax)

      #Printing into frames
      cv2.putText(frame,ymin_str, (50, 50),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
      cv2.putText(frame,ymax_str, (50, 70),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
      cv2.putText(frame,xmin_str, (50, 90),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
      cv2.putText(frame,xmax_str, (50, 110),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
                 

      try:

        for index, value in enumerate(classes[0]):
          object_dict = {}
          object_dict[(category_index.get(value)).get('name').encode('utf8')] = \
                            scores[0, index]
          for ind,val in object_dict.items():
            if(val > 0.7):
              print(ind)
              class_str = ind
      except:
        pass


      print(scores.max())
     
      print(class_str)
     
      
      
      #If prediction accuracy is 80% then do the following
      if scores.max() > 0.8:
        print("inprox")
        print(ymax , down_boundary_right[1])
        #If object is near, then give the warning
        if ymax >= down_boundary_right[1]:
          cv2.putText(frame,' WARNING! %s is near'%(class_str), (300, 600),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),2)	


      print("left_boundary[0],right_boundary[0] :", left_boundary[0], right_boundary[0])

      print("left_boundary[1],right_boundary[1] :", left_boundary[1], right_boundary[1])

      print("xmin, xmax :", xmin, xmax)
      print("ymin, ymax :", ymin, ymax)
      
      #Navigation algorithm
      if scores.max() > 0.8:
        print("inif")

        if(xmin >= left_boundary[0]):
          curr_navigation = 1
          print(curr_navigation,prev_navigation)
          if(curr_navigation == prev_navigation):
            cv2.putText(frame,"MOVE LEFT", (300, 100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),2)

        elif(xmax <= right_boundary[0]):
          curr_navigation = 2
          print(curr_navigation)
          
          if(curr_navigation == prev_navigation):
            cv2.putText(frame,"MOVE RIGHT", (300, 100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),2)

        elif(xmin <= left_boundary[0] and xmax >= right_boundary[0]):
          curr_navigation = 3
          print(curr_navigation,prev_navigation)
          if(curr_navigation == prev_navigation):
            cv2.putText(frame,"STOP", (300, 100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),2)
        
      prev_navigation = curr_navigation


#       cv2.line(frame,tuple(left_boundary),tuple(left_boundary_top), (255, 0, 0), 5)
#       cv2.line(frame,tuple(right_boundary),tuple(right_boundary_top), (255, 0, 0), 5)
#       cv2.line(frame,tuple(up_boundary_left),tuple(up_boundary_right), (255, 0, 0), 5)
#       cv2.line(frame,tuple(down_boundary_left),tuple(down_boundary_right), (255, 0, 0), 5)

      out.write(frame)
except:
  pass




720 1280
b'person'
0.76333547
b'person'
left_boundary[0],right_boundary[0] : 512 768
left_boundary[1],right_boundary[1] : 684 684
xmin, xmax : 487 529
ymin, ymax : 163 280
720 1280
b'person'
0.82865703
b'person'
inprox
278 503
left_boundary[0],right_boundary[0] : 512 768
left_boundary[1],right_boundary[1] : 684 684
xmin, xmax : 487 529
ymin, ymax : 163 278
inif
2
720 1280
b'person'
0.83983684
b'person'
inprox
282 503
left_boundary[0],right_boundary[0] : 512 768
left_boundary[1],right_boundary[1] : 684 684
xmin, xmax : 488 529
ymin, ymax : 164 282
inif
2
720 1280
b'person'
0.8080597
b'person'
inprox
286 503
left_boundary[0],right_boundary[0] : 512 768
left_boundary[1],right_boundary[1] : 684 684
xmin, xmax : 481 528
ymin, ymax : 161 286
inif
2
720 1280
0.68700707

left_boundary[0],right_boundary[0] : 512 768
left_boundary[1],right_boundary[1] : 684 684
xmin, xmax : 484 526
ymin, ymax : 166 286
720 1280
b'person'
0.7278362
b'person'
left_boundary[0],right_boundary[0] : 512 768
left_bound